In [22]:
import cv2
import os
import numpy as np
import mediapipe as mp

import csv

In [23]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [24]:
#Bisogna avere installata la libreria mediapipe
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [25]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task"
response = requests.get(url)

with open("hand_landmarker.task", "wb") as f:
    f.write(response.content)

print("File downloaded successfully.")

File downloaded successfully.


In [26]:
#Creo un oggetto Hand Landmarker
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

In [35]:
def video_to_data(video_path,filename):
# Verify the video path exists
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"The video path {video_path} does not exist.")

# Load the video
    #the file names of the videos specify the label: it's the last number before .mp4
    label = video_path[-5]
    cap = cv2.VideoCapture(video_path)
    mani=0
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Applico il detector di mediapipe e ottengo i risultati
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        detection_result = detector.detect(image)

        hand_landmarks_list = detection_result.hand_landmarks
        handedness_list = detection_result.handedness
        
        for idx in range(len(hand_landmarks_list)):
            hand_landmarks = hand_landmarks_list[idx]
            handedness = handedness_list[idx][0].index
            #ignoro la mano sinistra
            #if handedness == 1:
                #break
            x_coordinates = [landmark.x for landmark in hand_landmarks]
            y_coordinates = [landmark.y for landmark in hand_landmarks]
            z_coordinates = [landmark.z for landmark in hand_landmarks]
            mani+=1
            
            with open(filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                for idx in range(len(hand_landmarks_list)):
                # Write vector and label in a row
                    writer.writerow([handedness] + x_coordinates + y_coordinates + z_coordinates + [label])
            

            
        
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
    print("finito")

#print(mani)


In [36]:
def videos_to_data(folder_path, output_csv):
    video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]  # List all .mp4 videos
    
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        video_to_data(video_path, output_csv)

In [37]:
videos_to_data('D:/progetto_video/videos3', 'D:\progetto_video\dataset\dataset_nuovo_sx.csv')

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Marco\AppData\Local\Temp\ipykernel_8832\2011620950.py:1: SyntaxWarning: invalid escape sequence '\p'
  videos_to_data('D:/progetto_video/videos3', 'D:\progetto_video\dataset\dataset_nuovo_sx.csv')


finito
finito
finito
finito
finito
finito


In [30]:
import csv
with open('D:\progetto_video\dataset\dataset.csv', 'r') as file:
    reader = csv.reader(file)
    lista = []
    for row in reader:
        # Each row is a list of values (strings)
        lista.append(row[-1])


<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Marco\AppData\Local\Temp\ipykernel_8832\2870159594.py:2: SyntaxWarning: invalid escape sequence '\p'
  with open('D:\progetto_video\dataset\dataset.csv', 'r') as file:


In [31]:
import numpy as np
lista = [int(i) for i in lista]
lista = np.array(lista)
labels = []
for i in range(1,8):
    labels.append(np.sum(lista==i))

In [32]:
print(labels)

[991, 1075, 1145, 1240, 1156, 1081, 1137]


In [9]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('D:\progetto_video\dataset\dataset.csv')

# Define the label you want to remove and the label to change
label_to_remove = '7'
old_label = 8
new_label = 7

# Remove rows where the last column matches the label_to_remove
df_cleaned = df[df.iloc[:, -1] != label_to_remove]

# Replace the old label with the new label in the last column
df_cleaned.iloc[:, -1] = df_cleaned.iloc[:, -1].replace(old_label, new_label)

# Overwrite the original file with the cleaned and updated data
df_cleaned.to_csv('D:\progetto_video\dataset\dataset.csv', index=False)

print("Rows with the label removed and label 8 changed to 7 successfully.")


<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:18: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:18: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Marco\AppData\Local\Temp\ipykernel_8832\1644814952.py:4: SyntaxWarning: invalid escape sequence '\p'
  df = pd.read_csv('D:\progetto_video\dataset\dataset.csv')
C:\Users\Marco\AppData\Local\Temp\ipykernel_8832\1644814952.py:18: SyntaxWarning: invalid escape sequence '\p'
  df_cleaned.to_csv('D:\progetto_video\dataset\dataset.csv', index=False)


Rows with the label removed and label 8 changed to 7 successfully.


In [21]:
df = pd.read_csv('D:\progetto_video\dataset\dataset.csv')
df_cleaned = df[df.iloc[:, -1] != label_to_remove]
df_cleaned.to_csv('D:\progetto_video\dataset\dataset.csv', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Marco\AppData\Local\Temp\ipykernel_8832\3221072774.py:1: SyntaxWarning: invalid escape sequence '\p'
  df = pd.read_csv('D:\progetto_video\dataset\dataset.csv')
C:\Users\Marco\AppData\Local\Temp\ipykernel_8832\3221072774.py:3: SyntaxWarning: invalid escape sequence '\p'
  df_cleaned.to_csv('D:\progetto_video\dataset\dataset.csv', index=False)
